In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from torchmetrics import MeanMetric
from tqdm import tqdm
import sklearn
import cv2

from semsegcluster.eval_munkres import get_measurements, measure_from_confusion_matrix
import semsegcluster.data.scannet
from semsegcluster.settings import EXP_OUT
from semsegcluster.data.nyu_depth_v2 import TRAINING_LABEL_NAMES, NYU40_COLORS
import detectron2 as det2
import detectron2.utils.visualizer
from semsegcluster.sacred_utils import get_incense_loader
from deeplab.oaisys_utils import OAISYS_LABELS, data_converter_rugd

In [ ]:
det2.utils.colormap.random_color(rgb=True).astype(np.uint8)

### Visualize prediction and inlier/outlier

In [ ]:
skip_frames = 19
uncert_treshold = -6.6
subset = 'oaisys_trajectory3'
uncert = 'maxlogit-pp'
postfix = ''
stop_index = None
OAISYS_LABELS[15] = 'Sand'#'Grass-Field'
subsample = 1000
num_classes = 11

directory = os.path.join(EXP_OUT, f'oaisys_inference/{subset}/deeplab')
data = tfds.load(f'{subset}', split='validation')

m = det2.data.Metadata()
m.stuff_classes = OAISYS_LABELS + [f'c{i}' for i in range(80)]
m.stuff_colors = NYU40_COLORS + NYU40_COLORS + NYU40_COLORS
m_bool = det2.data.Metadata()
m_bool.stuff_classes = ['outlier', 'inlier']
m_bool.stuff_colors = NYU40_COLORS
for idx, blob in enumerate(data):
  # if idx%skip_frames != 0:
  #   continue
  if idx != 170:
    continue
  if stop_index is not None and idx>stop_index:
    break
  image = blob['image']
  label = blob['label']
  image, label = data_converter_rugd(image=image, label=label)
  image = tf.transpose(image, perm=[1, 2, 0]).numpy()
  label = label.numpy()
  pred = np.load(os.path.join(directory, f'{idx}{postfix}_pred.npy')).squeeze()
  loaded_label = np.load(os.path.join(directory, f'{idx}{postfix}_label.npy')).squeeze()
  uncertainty = np.load(os.path.join(directory, f'{idx}{postfix}_{uncert}.npy')).squeeze()
  # gmm_pred = np.load(os.path.join(directory, f'{idx:06d}_gmm_pred_{subsample}_{num_classes}.npy')).squeeze()
  # gmm_max_logit = np.load(os.path.join(directory, f'{idx:06d}_gmm_max_log_prob_{subsample}_{num_classes}.npy')).squeeze()
  ensemble_pred = np.load(os.path.join(directory, f'{idx}_ensemble_pred.npy')).squeeze()
  ensemble_max_logit = np.load(os.path.join(directory, f'{idx}_ensemble_{uncert}.npy')).squeeze()
  _, axs = plt.subplots(1, 6, figsize=(30, 20))
  axs[0].imshow(image)
  axs[0].axis('off')
  axs[0].set_title(f'image')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[1].imshow(vis.draw_sem_seg(label).get_image())
  axs[1].axis('off')
  axs[1].set_title(f'label')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[2].imshow(vis.draw_sem_seg(pred).get_image())
  axs[2].axis('off')
  axs[2].set_title(f'prediction ')
  vis = det2.utils.visualizer.Visualizer(image*255, m_bool, scale=0.4)
  axs[3].imshow(vis.draw_sem_seg(uncertainty<uncert_treshold).get_image())
  axs[3].axis('off')
  axs[3].set_title(f'inlier vs outlier, treshold {uncert_treshold}')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[4].imshow(vis.draw_sem_seg(ensemble_pred).get_image())
  axs[4].axis('off')
  axs[4].set_title(f'ensemble prediction')
  vis = det2.utils.visualizer.Visualizer(image*255, m_bool, scale=0.4)
  axs[5].imshow(vis.draw_sem_seg(ensemble_max_logit<-4.7).get_image())
  axs[5].axis('off')
  axs[5].set_title(f'ensemble max_logit inlier/outlier {idx}')
  plt.show()




  

### Visualize Mapping predictions and inliers/outliers

In [ ]:
skip_frames = 40
uncert_treshold = -4.7
subset = 'oaisys_trajectory3'
directory = os.path.join(EXP_OUT, f'oaisys_inference/{subset}/deeplab')
inlier = 'pseudolabel-pred'
#outlier = 'dbscansegandimgnet3027'
uncert = 'maxlogit-pp'
data = tfds.load(f'{subset}', split='validation')

m = det2.data.Metadata()
m.stuff_classes = OAISYS_LABELS + [f'c{i}' for i in range(80)]
m.stuff_colors = NYU40_COLORS + NYU40_COLORS + NYU40_COLORS
m_bool = det2.data.Metadata()
m_bool.stuff_classes = ['outlier', 'inlier']
m_bool.stuff_colors = NYU40_COLORS
for idx, blob in enumerate(data):
  if idx < 240 or idx > 280:
    continue
  # if idx%skip_frames != 0:
  #   continue
  image = blob['image']
  label = blob['label']
  image, label = data_converter_rugd(image, label)
  image = tf.transpose(image, perm=[1, 2, 0]).numpy()
  label = label.numpy()
  pred = np.load(os.path.join(directory, f'{idx}_pred.npy')).squeeze()
  # uncertainty = np.load(os.path.join(directory, f'{idx}_{uncert}.npy')).squeeze()
  rendered_pred = np.load(os.path.join(directory, f'{idx:06d}_pseudolabel-pred.npy')).squeeze()
  # rendered_uncertainty = np.load(os.path.join(directory, f'{idx:06d}_pseudolabel-{uncert}.npy')).squeeze()
  # i = np.load(os.path.join(directory, f'{frame}_{inlier}.npy')).squeeze()
  # #o = np.load(os.path.join(directory, f'{frame}_{outlier}.npy')).squeeze() + 40
  # #train_signal = i
  # #train_signal[u > -3] = o[u > -3]
  _, axs = plt.subplots(1, 4, figsize=(30, 20))
  axs[0].imshow(image)
  axs[0].axis('off')
  axs[0].set_title(f'image')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[1].imshow(vis.draw_sem_seg(label).get_image())
  axs[1].axis('off')
  axs[1].set_title(f'label')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[2].imshow(vis.draw_sem_seg(pred).get_image())
  axs[2].axis('off')
  axs[2].set_title(f'prediction ')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[3].imshow(vis.draw_sem_seg(rendered_pred).get_image())
  axs[3].axis('off')
  axs[3].set_title(f'rendered prediction')
  # vis = det2.utils.visualizer.Visualizer(image*255, m_bool, scale=0.4)
  # axs[4].imshow(vis.draw_sem_seg(uncertainty<uncert_treshold).get_image())
  # axs[4].axis('off')
  # axs[4].set_title(f'inlier vs outlier {idx}, treshold {uncert_treshold}')
  # vis = det2.utils.visualizer.Visualizer(image*255, m_bool, scale=0.4)
  # axs[5].imshow(vis.draw_sem_seg(rendered_uncertainty<uncert_treshold).get_image())
  # axs[5].axis('off')
  # axs[5].set_title(f'rendered inlier vs outlier {idx}, treshold {uncert_treshold}')
  # vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  # #axs[2].imshow(u, vmin=-5, vmax=3)
  # axs[3].imshow(u > -3)
  # axs[3].axis('off')
  plt.show()

### Visualize Prediction, Adapted Prediction

In [ ]:
skip_frames = 16
subset = 'oaisys_trajectory3'
training = '89'
pseudolabel = 'ground-truth'

directory = os.path.join(EXP_OUT, f'oaisys_inference/{subset}/deeplab')
data = tfds.load(f'{subset}', split='validation')

m = det2.data.Metadata()
m.stuff_classes = OAISYS_LABELS + [f'c{i}' for i in range(80)]
m.stuff_colors = NYU40_COLORS + NYU40_COLORS + NYU40_COLORS
for idx, blob in enumerate(data):
  if idx%skip_frames != 0:
    continue
  image = blob['image']
  label = blob['label']
  image, label = data_converter_rugd(image=image, label=label)
  image = tf.transpose(image, perm=[1, 2, 0]).numpy()
  label = label.numpy()
  pred = np.load(os.path.join(directory, f'{idx}_pred.npy')).squeeze()
  try:
    adapted_pred = np.load(os.path.join(directory, f'{idx:06d}_pred{training}-{pseudolabel}.npy')).squeeze()
  except FileNotFoundError:
    adapted_pred = cv2.imread(os.path.join(directory, f'{idx:06d}_pred{training}-{pseudolabel}.png'),
                         cv2.IMREAD_ANYDEPTH).squeeze().astype(np.int32)
  _, axs = plt.subplots(1, 4, figsize=(30, 20))
  axs[0].imshow(image)
  axs[0].axis('off')
  axs[0].set_title(f'image')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[1].imshow(vis.draw_sem_seg(label).get_image())
  axs[1].axis('off')
  axs[1].set_title(f'label')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[2].imshow(vis.draw_sem_seg(pred).get_image())
  axs[2].axis('off')
  axs[2].set_title(f'prediction')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[3].imshow(vis.draw_sem_seg(adapted_pred-1).get_image())
  axs[3].axis('off')
  axs[3].set_title(f'adapted_prediction')
  plt.show()

### Visulaize prediction, uncertainty, merged labels

In [ ]:
skip_frames = 20
uncert_treshold = -4.5
subset = 'oaisys_trajectory_grass'
inlier = 'ensemble_pred'
outlier = 'seg122'
uncert = 'ensemble_maxlogit-pp'


directory = os.path.join(EXP_OUT, f'oaisys_inference/{subset}/deeplab')
data = tfds.load(f'{subset}', split='validation')

m = det2.data.Metadata()
m.stuff_classes = OAISYS_LABELS + [f'c{i}' for i in range(80)]
m.stuff_colors = NYU40_COLORS + NYU40_COLORS + NYU40_COLORS
m_bool = det2.data.Metadata()
m_bool.stuff_classes = ['outlier', 'inlier']
m_bool.stuff_colors = NYU40_COLORS
for idx, blob in enumerate(data):
  if idx%skip_frames != 0:
    continue
  image = blob['image']
  label = blob['label']
  image, label = data_converter_rugd(image=image, label=label)
  image = tf.transpose(image, perm=[1, 2, 0]).numpy()
  label = label.numpy()
  pred = np.load(os.path.join(directory, f'{idx}_pred.npy')).squeeze()
  uncertainty = np.load(os.path.join(directory, f'{idx}_{uncert}.npy')).squeeze()
  try:
    merged = np.load(os.path.join(directory, f'{idx:06d}_merged-{inlier}-{outlier}.npy')).squeeze()
  except FileNotFoundError:
    merged = cv2.imread(os.path.join(directory, f'{idx:06d}_merged-{inlier}-{outlier}.png'),
                         cv2.IMREAD_ANYDEPTH).squeeze().astype(np.int32)
  _, axs = plt.subplots(1, 5, figsize=(30, 20))
  axs[0].imshow(image)
  axs[0].axis('off')
  axs[0].set_title(f'image {idx}')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[1].imshow(vis.draw_sem_seg(label).get_image())
  axs[1].axis('off')
  axs[1].set_title(f'label {idx}')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[2].imshow(vis.draw_sem_seg(pred).get_image())
  axs[2].axis('off')
  axs[2].set_title(f'prediction {idx}')
  vis = det2.utils.visualizer.Visualizer(image*255, m_bool, scale=0.4)
  axs[3].imshow(vis.draw_sem_seg(uncertainty<uncert_treshold).get_image())
  axs[3].axis('off')
  axs[3].set_title(f'inlier vs outlier {idx}, treshold {uncert_treshold}')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[4].imshow(vis.draw_sem_seg(merged).get_image())
  axs[4].axis('off')
  axs[4].set_title(f'merged {idx}')
  plt.show()

### Visuluaize image, label, inliner/outlier, prediction, cluster, pseudolabel

In [ ]:
skip_frames = 19
subset = 'oaisys_trajectory3'
outlier_class = 'Sand'
training = '402'
clust = 'seg107'
inlier='ensemble_pred'
uncert = 'ensemble_maxlogit-pp'
uncert_treshold = -4.7
pseudolabel = f'merged-{inlier}-{clust}'

directory = os.path.join(EXP_OUT, f'oaisys_inference/{subset}/deeplab')
data = tfds.load(f'{subset}', split='validation')
OAISYS_LABELS[15] = 'Grass-Field'
m = det2.data.Metadata()
m.stuff_classes = OAISYS_LABELS[0:11] + [f'c{i}' for i in range(80)]
m.stuff_colors = NYU40_COLORS + NYU40_COLORS + NYU40_COLORS
m_label = det2.data.Metadata()
m_label.stuff_classes = OAISYS_LABELS[0:11] + [f'c{i}' for i in range(80)]
m_label.stuff_colors = NYU40_COLORS + NYU40_COLORS + NYU40_COLORS
m_label.stuff_classes[15] = outlier_class
m_bool = det2.data.Metadata()
m_bool.stuff_classes = ['outlier', 'inlier']
m_bool.stuff_colors = NYU40_COLORS
for idx, blob in enumerate(data):
  if idx%skip_frames != 0:
    continue
  # if idx not in [288, 170]:
  #   continue
  image = blob['image']
  label = blob['label']
  image, label = data_converter_rugd(image=image, label=label)
  image = tf.transpose(image, perm=[1, 2, 0]).numpy()
  label = label.numpy()
  pred = np.load(os.path.join(directory, f'{idx}_{inlier}.npy')).squeeze()
  uncertainty = np.load(os.path.join(directory, f'{idx}_{uncert}.npy')).squeeze()
  cluster = np.load(os.path.join(directory, f'{idx:06d}_{clust}.npy')).squeeze()
  merged = cv2.imread(os.path.join(directory, f'{idx:06d}_merged-{inlier}-{clust}_{uncert_treshold}.png'),
                         cv2.IMREAD_ANYDEPTH).squeeze().astype(np.int32)
  try:
    adapted_pred = np.load(os.path.join(directory, f'{idx:06d}_pred{training}-{pseudolabel}.npy')).squeeze()
  except FileNotFoundError:
    adapted_pred = cv2.imread(os.path.join(directory, f'{idx:06d}_pred{training}-{pseudolabel}.png'),
                         cv2.IMREAD_ANYDEPTH).squeeze().astype(np.int32)
  # try:
  #   adapted_pred_gt = np.load(os.path.join(directory, f'{idx:06d}_pred89-ground-truth.npy')).squeeze()
  # except FileNotFoundError:
  #   adapted_pred_gt = cv2.imread(os.path.join(directory, f'{idx:06d}_pred89-ground-truth.png'),
  #                        cv2.IMREAD_ANYDEPTH).squeeze().astype(np.int32)
  _, axs = plt.subplots(1, 7, figsize=(30, 20))
  axs[0].imshow(image)
  axs[0].axis('off')
  axs[0].set_title(f'Image')
  vis = det2.utils.visualizer.Visualizer(image*255, m_label, scale=0.4)
  axs[1].imshow(vis.draw_sem_seg(label).get_image())
  axs[1].axis('off')
  axs[1].set_title(f'Label')
  vis = det2.utils.visualizer.Visualizer(image*255, m_bool, scale=0.4)
  axs[2].imshow(vis.draw_sem_seg(uncertainty<uncert_treshold).get_image())
  axs[2].axis('off')
  axs[2].set_title(f'OOD Detection')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[3].imshow(vis.draw_sem_seg(pred).get_image())
  axs[3].axis('off')
  axs[3].set_title(f'Deep Ensemble Prediction ')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[4].imshow(vis.draw_sem_seg(cluster+12).get_image())
  axs[4].axis('off')
  axs[4].set_title(f'Deeplabv3 Clustering')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[5].imshow(vis.draw_sem_seg(merged).get_image())
  axs[5].axis('off')
  axs[5].set_title(f'Pseudolabels')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[6].imshow(vis.draw_sem_seg(adapted_pred).get_image())
  axs[6].axis('off')
  axs[6].set_title(f'Adapted Prediction')
  # vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  # axs[7].imshow(vis.draw_sem_seg(adapted_pred_gt).get_image())
  # axs[7].axis('off')
  # axs[7].set_title(f'adapted_prediction')
  plt.show()

In [ ]:
skip_frames = 1
subset = 'rugd_sand'
outlier_class = 'Sand'
training = '456'
clust = 'dino454'
inlier='ensemble_pred'
uncert = 'ensemble_maxlogit-pp'
uncert_treshold = -4.39
pseudolabel = 'merged-ensemble_pred-dino454_-4.39'

directory = os.path.join(EXP_OUT, f'oaisys_inference/{subset}/deeplab')
data = tfds.load(f'{subset}', split='validation')
OAISYS_LABELS[15] = 'Grass-Field'
m = det2.data.Metadata()
m.stuff_classes = OAISYS_LABELS[0:11] + [f'c{i}' for i in range(80)]
m.stuff_colors = NYU40_COLORS + NYU40_COLORS + NYU40_COLORS
m_label = det2.data.Metadata()
m_label.stuff_classes = OAISYS_LABELS[0:11] + [f'c{i}' for i in range(80)]
m_label.stuff_colors = NYU40_COLORS + NYU40_COLORS + NYU40_COLORS
m_label.stuff_classes[15] = outlier_class
m_bool = det2.data.Metadata()
m_bool.stuff_classes = ['outlier', 'inlier']
m_bool.stuff_colors = NYU40_COLORS
for idx, blob in enumerate(data):
  # if idx%skip_frames != 0:
  #   continue
  if idx != 6:
    continue
  image = blob['image']
  label = blob['label']
  image, label = data_converter_rugd(image=image, label=label)
  image = tf.transpose(image, perm=[1, 2, 0]).numpy()
  label = label.numpy()
  # if not np.any(label == 15):
  #   continue
  pred = np.load(os.path.join(directory, f'{idx}_{inlier}.npy')).squeeze()
  uncertainty = np.load(os.path.join(directory, f'{idx}_{uncert}.npy')).squeeze()
  cluster = np.load(os.path.join(directory, f'{idx:06d}_{clust}.npy')).squeeze()
  merged = cv2.imread(os.path.join(directory, f'{idx:06d}_merged-{inlier}-{clust}_{uncert_treshold}.png'),
                         cv2.IMREAD_ANYDEPTH).squeeze().astype(np.int32)
  # if not os.path.exists(os.path.join(directory, f'{idx:06d}_pred{training}-{pseudolabel}.png')):
  #   break
  adapted_pred = cv2.imread(os.path.join(directory, f'{idx:06d}_pred{training}-{pseudolabel}.png'),
                         cv2.IMREAD_ANYDEPTH).squeeze().astype(np.int32)
  # # try:
  # #   adapted_pred_gt = np.load(os.path.join(directory, f'{idx:06d}_pred89-ground-truth.npy')).squeeze()
  # # except FileNotFoundError:
  # #   adapted_pred_gt = cv2.imread(os.path.join(directory, f'{idx:06d}_pred89-ground-truth.png'),
  # #                        cv2.IMREAD_ANYDEPTH).squeeze().astype(np.int32)
  _, axs = plt.subplots(1, 7, figsize=(30, 20))
  axs[0].imshow(image)
  axs[0].axis('off')
  axs[0].set_title(f'Image')
  vis = det2.utils.visualizer.Visualizer(image*255, m_label, scale=0.4)
  axs[1].imshow(vis.draw_sem_seg(label).get_image())
  axs[1].axis('off')
  axs[1].set_title(f'Label')
  vis = det2.utils.visualizer.Visualizer(image*255, m_bool, scale=0.4)
  axs[2].imshow(vis.draw_sem_seg(uncertainty<uncert_treshold).get_image())
  axs[2].axis('off')
  axs[2].set_title(f'OOD Detection')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[3].imshow(vis.draw_sem_seg(pred).get_image())
  axs[3].axis('off')
  axs[3].set_title(f'Deep Ensemble Prediction')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[4].imshow(vis.draw_sem_seg(cluster+12).get_image())
  axs[4].axis('off')
  axs[4].set_title(f'Dino Clutering')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[5].imshow(vis.draw_sem_seg(merged).get_image())
  axs[5].axis('off')
  axs[5].set_title(f'Pseudolabel')
  vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  axs[6].imshow(vis.draw_sem_seg(adapted_pred).get_image())
  axs[6].axis('off')
  axs[6].set_title(f'Adapted Prediction')
  # # vis = det2.utils.visualizer.Visualizer(image*255, m, scale=0.4)
  # # axs[7].imshow(vis.draw_sem_seg(adapted_pred_gt).get_image())
  # # axs[7].axis('off')
  # # axs[7].set_title(f'adapted_prediction')
  plt.show()

In [ ]:
subset = 'rugd'

count = 0
num_pixels = 0
data = tfds.load(f'{subset}', split='validation')
for blob in tqdm(data):
    image = blob['image']
    label = blob['label']
    image, label = data_converter_rugd(image=image, label=label)
    label = label.numpy()
    if np.any(label == 15):
        count += 1
        num_pixels += np.sum(label == 15)
print(f'found {count} frames with sand')
print(f'found {num_pixels} pixels with sand')
print(f'found {num_pixels/count} pixels with sand per frame')
print(f'found {num_pixels/(count*480*640)} pixels with sand per frame')